## Thinking1：常用的路径规划算法有哪些?
### answer： Dijkstra算法，Floyd算法。

## Thinking2:推荐系统的架构是怎样的？
### answer：1、从数据库或缓存中拿到用户行为数据，通过分析不同行为，生成当前用户的特征向量；2、将用户的特征向量通过特征-物品相关举证转换为初始推荐物品列表；3、对初始的推荐列表进行过滤、排名等处理，生成最终的推荐结果。

## Thinking3:你都了解推荐系统中的哪些常用算法？原理是怎样的？
### answer：ALS、SVD、FM/DeepFM、GBDT。

In [44]:
#Action 1：基于高德地图的路径规划
import pandas as pd
import requests
from bs4 import BeautifulSoup

header={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
"""
#根据request_url得到soup
def get_page_content(request_url):
    #得到页面的内容
    html = requests.get(request_url, headers=header, timeout=10)
    content = html.text
    #print(content)
    soup = BeautifulSoup(content, 'html.parser', from_encoding='utf-8')
    return soup

request_url = 'https://ditie.mapbar.com/beijing_line/'
soup = get_page_content(request_url)

#name,site
df = pd.DataFrame(columns = ['name', 'site'])
subways = soup.find_all('div', class_='station')
for subway in subways:
    route_name = subway.find('strong', class_='bolder').text
    routes = subway.find('ul')
    routes = routes.find_all('a')
    #对于所有的route都需要保存name
    for route in routes:
        temp = {'name': route.text, 'site': route_name}
        df = df.append(temp, ignore_index=True)
        #print(temp)
    print(route_name)
df['city'] = '北京'
print(df)
df.to_csv('subway.csv', index=False, encoding='utf-8')
"""

import re
#通过keyword, city得到指定的location
def get_location(keyword, city):
    #request_url = 'http://restapi.amap.com/v3/place/text?key=35fc3e2f362fa1eeec425f4468f2452b&keywords=' + keyword + ',' +'&types=&city=' + city +'&children=1&offset=1&page=1&extensions=all'
    request_url = 'http://restapi.amap.com/v3/place/text?key=1be5cb2b4713e7519e5fa36e6996bba3&keywords=' + keyword + ',' +'&types=&city=' + city +'&children=1&offset=1&page=1&extensions=all'
    #request_url = 'http://restapi.amap.com/v3/place/text?key=35fc3e2f362fa1eeec425f4468f2452b&keywords=' + keyword +'&types=&city=' + city +'&children=1&offset=1&page=1&extensions=all'
    data = requests.get(request_url, headers=header, timeout=10)
    data.encoding = 'utf-8'
    data = data.text
   
    #"location":"116.306319,40.052299"
    #后边多了一个？，表示懒惰模式
    #.*具有贪婪模式，匹配到不能匹配为止
    #.*?取消贪婪，一个匹配以后，就继续后面的正则
    pattern = 'location":"(.*?),(.*?)"'
    #得到经纬度
    #print(request_url)
    result = re.findall(pattern, data)
    try:
        return result[0][0], result[0][1]
    except:
        return get_location(keyword.replace('站', ''), city)

df = pd.read_csv('./subway.csv', index_col=None)
print(df.head())
df['longitude'], df['latitude'] = None, None
for index, row in df.iterrows():
    name, city = row['name'], row['city']
    #print(name, city)
    longitude, latitude = get_location(name, city)
    df.iloc[index]['longitude'] = longitude
    df.iloc[index]['latitude'] = latitude
    print(longitude, latitude)
df.to_csv('subway.csv', index=False)
#get_location('五道口','北京')
 

    name   site city   longitude   latitude
0    四惠站  地铁八通线   北京  116.498489  39.908279
1   四惠东站  地铁八通线   北京  116.514954  39.908051
2   高碑店站  地铁八通线   北京  115.858901  39.328727
3  传媒大学站  地铁八通线   北京  116.554931  39.909548
4    双桥站  地铁八通线   北京  116.569498  39.901066
116.498489 39.908279
116.514954 39.908051
115.858901 39.328727
116.554931 39.909548
116.569498 39.901066
116.598786 39.908696
116.617844 39.906559
116.636748 39.904070
116.646183 39.893199
116.662943 39.888520
116.669083 39.883915
116.679099 39.875796
116.687215 39.871679
116.306169 40.052307
116.294575 40.094705
116.313248 40.104775
116.293671 40.130836
116.265245 40.125131
116.279991 40.164629
116.287159 40.207766
116.282712 40.221261
116.262840 40.221475
116.194657 40.190277
116.207894 40.239493
116.194237 40.244382
116.125346 39.723503
116.140429 39.723478
116.155903 39.723547
116.176119 39.722872
116.183056 39.729827
116.180550 39.746964
116.189793 39.760327
116.212569 39.763968
116.218561 39.794460
116.291802 39.807984
1

In [45]:
get_location('五道口','北京')

('116.337581', '39.993138')

In [56]:
#路径规划
import pandas as pd
import re, requests

header={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
#计算两点之间的距离
def compute_distance(longitude1, latitude1, longitude2, latitude2):
    #request_url = 'http://restapi.amap.com/v3/distance?key=35fc3e2f362fa1eeec425f4468f2452b&origins=' + str(longitude1) + ',' +str(latitude1) +'&destination=' + str(longitude2) +',' +str(latitude2)+'&type=1'
    request_url = 'http://restapi.amap.com/v3/distance?key=1be5cb2b4713e7519e5fa36e6996bba3&origins=' + str(longitude1) + ',' +str(latitude1) +'&destination=' + str(longitude2) +',' +str(latitude2)+'&type=1'
    data = requests.get(request_url, headers=header, timeout=10)
    data.encoding = 'utf-8'
    data = data.text
    #"distance":"2646","duration":"660"
    pattern = "distance\":\"(.*?)\",\"duration\":\"(.*?)\""
    result = re.findall(pattern, data)
    #返回距离result[0][0]；返回时间result[0][1]
    return result[0][0]
# 五道口的坐标(116.337581,39.993138)
# 知春路的坐标(116.339941,39.976228)
print (compute_distance(116.337581,39.993138, 116.339941,39.976228))
'''
#数据加载
data = pd.read_csv('./subway.csv', index_col=None)
#print(data)

from collections import defaultdict
#创建图中两点之间的最短距离
graph = defaultdict(dict)

for i in range(data.shape[0]):
    site1 = data.iloc[i]['site']
    if i<data.shape[0]-1:
        site2 = data.iloc[i+1]['site']
    #如果是同一条线路
        if site1 == site2:
            longitude1, latitude1 = data.iloc[i]['longitude'], data.iloc[i]['latitude']
            longitude2, latitude2 = data.iloc[i+1]['longitude'], data.iloc[i+1]['latitude']
            name1, name2 = data.iloc[i]['name'], data.iloc[i+1]['name']
            distance = compute_distance(longitude1, latitude1, longitude2, latitude2)
            graph[name1][name2] = distance
            graph[name2][name1] = distance
            print(name1, name2, distance)

import pickle
output = open('graph.pkl', 'wb')
pickle.dump(graph, output)
print(graph)
'''
import pickle
file = open('graph.pkl', 'rb')
graph = pickle.load(file)
print(graph)

# 找到开销最小的节点
def find_lowest_cost_node(costs):
    #初始化数据
    lowest_cost = float('inf')
    lowest_cost_node = None
    #遍历所有节点
    for node in costs:
        #如果这个节点没有被处理
        if not node in processed:
            #如果当前节点的cost比已经存在的cost小，那么擂主更新，即更新这个节点为cost最小的节点
            if costs[node] < lowest_cost:
                lowest_cost = costs[node]
                lowest_cost_node = node
    return lowest_cost_node

# 找到最短路径,从end到start的完整路径
def find_shortest_path():
    node = end
    shortest_path = [end]
    while parents[node] != start:
        shortest_path.append(parents[node])
        node = parents[node]
    shortest_path.append(start)
    return shortest_path

# 计算图中从start到end的最短路径
def dijkstra():
    #查询到目前开销最小的节点
    node = find_lowest_cost_node(costs)
    print('当前cost最小节点：', node)
    #只要有cost最小的节点，就进行路径计算，如果所有节点都在processed中，就结束了
    while node is not None:
        #获取节点目前的cost
        cost = costs[node]
        #获取节点的邻居节点
        neighbors = graph[node]
        #遍历所有邻居，看是否可以通过node完成cost的更新
        for neighbor in neighbors.keys():
            #计算经过当前节点到达邻居节点的cost
            new_cost = cost + float(neighbors[neighbor])
            #如果通过node可以更新start->neighborde cost
            if neighbor not in costs or new_cost < costs[neighbor]:
                costs[neighbor] = new_cost
                parents[neighbor] = node
        #将当前节点标记为处理过
        processed.append(node)
        
        #找出接下来要处理的节点并循环
        node = find_lowest_cost_node(costs)
    #循环完毕说明所有节点都处理完毕
    shortest_path = find_shortest_path()
    shortest_path.reverse()
    print('从{}到{}的最短路径：{}'.format(start, end, shortest_path))

                

start = '五道口站'
end = '北京南站'   


# 创建节点的开销表，cost是指从start到该节点的距离
costs = {}
# 存储父节点的Hash表，用于记录路径
parents = {} 
parents[end] = None
# 记录处理过的节点list
processed = []

#获取节点的邻居系欸但
for node in graph[start].keys():
    costs[node] = float(graph[start][node])
    parents[node] = start
costs[end] = float('inf')

dijkstra()

2646
defaultdict(<class 'dict'>, {'四惠站': {'四惠东站': '4141', '大望路站': '4413'}, '四惠东站': {'四惠站': '4141', '高碑店站': '108555'}, '高碑店站': {'四惠东站': '108555', '传媒大学站': '111745'}, '传媒大学站': {'高碑店站': '111745', '双桥站': '4827'}, '双桥站': {'传媒大学站': '4827', '管庄站': '3648'}, '管庄站': {'双桥站': '3648', '八里桥站': '2411'}, '八里桥站': {'管庄站': '2411', '通州北苑站': '3275'}, '通州北苑站': {'八里桥站': '3275', '果园站': '1470'}, '果园站': {'通州北苑站': '1470', '九棵树站': '1445'}, '九棵树站': {'果园站': '1445', '梨园站': '2323'}, '梨园站': {'九棵树站': '2323', '临河里站': '2733'}, '临河里站': {'梨园站': '2733', '土桥站': '1674'}, '土桥站': {'临河里站': '1674'}, '西二旗站': {'生命科学园站': '6828', '上地站': '2870', '龙泽站': '6891'}, '生命科学园站': {'西二旗站': '6828', '朱辛庄站': '3478'}, '朱辛庄站': {'生命科学园站': '3478', '巩华城站': '6679', '育知路站': '2964'}, '巩华城站': {'朱辛庄站': '6679', '沙河站': '3888'}, '沙河站': {'巩华城站': '3888', '沙河高教园站': '7253'}, '沙河高教园站': {'沙河站': '7253', '南邵站': '6743'}, '南邵站': {'沙河高教园站': '6743', '北邵洼站': '1981'}, '北邵洼站': {'南邵站': '1981', '昌平东关站': '2742'}, '昌平东关站': {'北邵洼站': '2742', '昌平站': '11189'}, '昌平站': {'昌平东关站': '1118